# Vanguard

In [166]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
from statsmodels.stats.proportion import proportions_ztest, confint_proportions_2indep
import numpy as np
from scipy import stats
import statsmodels.formula.api as smf
from statsmodels.stats.power import NormalIndPower
from scipy.stats import chi2_contingency

In [ ]:
df_exp_clients = pd.read_csv("archivos/df_final_experiment_clients.txt")
df_web1 = pd.read_csv("archivos/df_final_web_data_pt_1.txt")
df_web2 = pd.read_csv("archivos/df_final_web_data_pt_2.txt")
df_demo = pd.read_csv("archivos/df_final_demo.txt")

### TAMAÑO DE CADA DATASET

In [168]:
#Filas y columnas de c/u
dfs = {
    "exp_clients": df_exp_clients,
    "web1": df_web1,
    "web2": df_web2,
    "demo": df_demo
}

for name, df in dfs.items():
    print(f"\n{name}")
    print("Shape:", df.shape)
    print("Columnas:", df.columns.tolist())


exp_clients
Shape: (70609, 2)
Columnas: ['client_id', 'Variation']

web1
Shape: (343141, 5)
Columnas: ['client_id', 'visitor_id', 'visit_id', 'process_step', 'date_time']

web2
Shape: (412264, 5)
Columnas: ['client_id', 'visitor_id', 'visit_id', 'process_step', 'date_time']

demo
Shape: (70609, 9)
Columnas: ['client_id', 'clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age', 'gendr', 'num_accts', 'bal', 'calls_6_mnth', 'logons_6_mnth']


In [169]:
#TODOS TIENEN EN COMÚN CLIENT_ID , ASÍ COMO VARIABLES CATEGORICAS Y NUMÉRICAS

In [170]:
df_exp_clients.info()
df_web1.info()
df_web2.info()
df_demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70609 entries, 0 to 70608
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  70609 non-null  int64 
 1   Variation  50500 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343141 entries, 0 to 343140
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   client_id     343141 non-null  int64 
 1   visitor_id    343141 non-null  object
 2   visit_id      343141 non-null  object
 3   process_step  343141 non-null  object
 4   date_time     343141 non-null  object
dtypes: int64(1), object(4)
memory usage: 13.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412264 entries, 0 to 412263
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   client_id     412264 no

Tipos de Variables y Valores Nulos

In [171]:
for name, df in dfs.items():
    print(f"\n {name}")
    display(df.info())


 exp_clients
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70609 entries, 0 to 70608
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  70609 non-null  int64 
 1   Variation  50500 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


None


 web1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343141 entries, 0 to 343140
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   client_id     343141 non-null  int64 
 1   visitor_id    343141 non-null  object
 2   visit_id      343141 non-null  object
 3   process_step  343141 non-null  object
 4   date_time     343141 non-null  object
dtypes: int64(1), object(4)
memory usage: 13.1+ MB


None


 web2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412264 entries, 0 to 412263
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   client_id     412264 non-null  int64 
 1   visitor_id    412264 non-null  object
 2   visit_id      412264 non-null  object
 3   process_step  412264 non-null  object
 4   date_time     412264 non-null  object
dtypes: int64(1), object(4)
memory usage: 15.7+ MB


None


 demo
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70609 entries, 0 to 70608
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   client_id         70609 non-null  int64  
 1   clnt_tenure_yr    70595 non-null  float64
 2   clnt_tenure_mnth  70595 non-null  float64
 3   clnt_age          70594 non-null  float64
 4   gendr             70595 non-null  object 
 5   num_accts         70595 non-null  float64
 6   bal               70595 non-null  float64
 7   calls_6_mnth      70595 non-null  float64
 8   logons_6_mnth     70595 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 4.8+ MB


None

#### Eliminando nulos 'Variation'

In [172]:
# Eliminando en Variation los nulos.
df_exp_clients = df_exp_clients.dropna(subset=["Variation"])

In [173]:
# Cambiando tipo de dato a la columna Client_id a str
datas = [df_exp_clients, df_web1, df_web2, df_demo]

for df in datas:
    df["client_id"] = (df["client_id"].astype(str).str.strip()
                       .str.replace(".0", "", regex=False))

/var/folders/m_/06pskt392pn6qvkq92krm8xm0000gn/T/ipykernel_34731/2826387553.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["client_id"] = (df["client_id"].astype(str).str.strip()


In [174]:
#verificando tamaños reales 
print("Exp únicos:", df_exp_clients["client_id"].nunique())
print("Web1 únicos:", df_web1["client_id"].nunique())
print("Web2 únicos:", df_web2["client_id"].nunique())

#Hacemos el concat de los dos datset de web y vemos cuantos valores unicos hay
df_web = pd.concat([df_web1, df_web2], ignore_index=True)
print("Web total únicos:", df_web["client_id"].nunique())

Exp únicos: 50500
Web1 únicos: 58391
Web2 únicos: 67430
Web total únicos: 120157


In [175]:
#verificamos que realmente coinciden los client_ids 
clients_exp = set(df_exp_clients["client_id"])
clients_web = set(df_web["client_id"])

inter = clients_exp & clients_web

print("Intersección:", len(inter))
print("Cobertura:", len(inter) / len(clients_exp))

Intersección: 50500
Cobertura: 1.0


100% de cobertura, todos los clientes en el experimento tienen al menos un registro en el dataset web

#### COMPORTAMIENTO

Lo mediremos primero dentro de cada dataset segun su naruraleza para finalmente unirlos. Se encuentran 2 tipos de comportamientos:
1.-De tipo historico; en este se puede revisar la intensidad digital historica, la frecuencia de contacto y el perfil financiero
2.-De eventos web; enseste se mide el comportamiento durante el experimento, se revisan las interacciones realses como no. de eventos y sesiones, funnel y conversión.

COMPORTAMIENTO HISTÓRICO
(df_demo)

In [176]:
df_demo

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6.0,73.0,60.5,U,2.0,45105.30,6.0,9.0
1,2304905,7.0,94.0,58.0,U,2.0,110860.30,6.0,9.0
2,1439522,5.0,64.0,32.0,U,2.0,52467.79,6.0,9.0
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0
...,...,...,...,...,...,...,...,...,...
70604,7993686,4.0,56.0,38.5,U,3.0,1411062.68,5.0,5.0
70605,8981690,12.0,148.0,31.0,M,2.0,101867.07,6.0,6.0
70606,333913,16.0,198.0,61.5,F,2.0,40745.00,3.0,3.0
70607,1573142,21.0,255.0,68.0,M,3.0,475114.69,4.0,4.0


nuestra variable logons_6_mnth, segmentarla en niveles de intensidad digital usando percentiles

### Clientes principales:

In [177]:
#'logon_segment', divide en percentil el client principal
df_demo["logon_segment"] = pd.qcut(
    df_demo["logons_6_mnth"],
    q=3,
    labels=["Low", "Medium", "High"]
)

In [178]:
df_demo["logon_segment"].value_counts(normalize=True)

logon_segment
Medium    0.382817
Low       0.374106
High      0.243077
Name: proportion, dtype: float64

Son jovenes o mayores? Nuevos o antiguos?

In [179]:
#perfil por segmento: edad del cliente, balance, cuentas supongo tiempo en años.
df_demo.groupby("logon_segment")[["clnt_age","clnt_tenure_yr","bal","num_accts"]].mean()

/var/folders/m_/06pskt392pn6qvkq92krm8xm0000gn/T/ipykernel_34731/4103272527.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_demo.groupby("logon_segment")[["clnt_age","clnt_tenure_yr","bal","num_accts"]].mean()


,clnt_age,clnt_tenure_yr,bal,num_accts
logon_segment,,,,
Low,44.742976,11.631503,94853.286136,2.124612
Medium,47.200747,12.222091,155163.365490,2.283404
High,47.862966,12.435198,216231.440784,2.413112


In [180]:
#son ligeramente mayores (~3 años más que el segmento low)
#no son los mas jovenes

#son mas antiguos (mayor tenure)
#Tienen un promedio casi 1 año mas de antiguedad que el segmento Low.

In [181]:
# Merge de demo y clientes
df_exp_demo = df_exp_clients.merge(
    df_demo[["client_id", "logon_segment"]],
    on="client_id",
    how="left")

In [182]:
# verificar que el experimento este balanceado con el cliente principal 
pd.crosstab(
    df_exp_demo["Variation"],
    df_exp_demo["logon_segment"],
    normalize="index")


logon_segment,Low,Medium,High
Variation,,,
Control,0.292047,0.361585,0.346368
Test,0.307667,0.358110,0.334224


In [183]:
#Opción A — Validar si esas diferencias son estadísticamente significativas
low = df_demo[df_demo["logon_segment"]=="Low"]["bal"]
high = df_demo[df_demo["logon_segment"]=="High"]["bal"]

stats.ttest_ind(low, high, equal_var=False)

TtestResult(statistic=-35.260585785493426, pvalue=7.222931639381394e-265, df=21632.67986111101)

In [184]:
#Esta balance

In [185]:
# Merge estático (nivel cliente):
df_static = df_exp_clients.merge(df_demo, on="client_id", how="left")

df_static["client_id"].nunique()
df_static.shape

(50500, 11)

In [186]:
df_static.isna().sum()

client_id            0
Variation            0
clnt_tenure_yr      12
clnt_tenure_mnth    12
clnt_age            13
gendr               12
num_accts           12
bal                 12
calls_6_mnth        12
logons_6_mnth       12
logon_segment       12
dtype: int64

In [187]:
#limpiando nulos
df_static = df_static.dropna(
    subset=["clnt_age","clnt_tenure_yr","bal","gendr","num_accts","logons_6_mnth"]
)

##### Creación df_master

In [188]:
#Merge df_static con web
df_master = df_static.merge(df_web, on="client_id", how="left")
df_master.shape

(321195, 15)

#### Limpieza df_master

In [189]:
df_master.isnull().sum()

client_id           0
Variation           0
clnt_tenure_yr      0
clnt_tenure_mnth    0
clnt_age            0
gendr               0
num_accts           0
bal                 0
calls_6_mnth        0
logons_6_mnth       0
logon_segment       0
visitor_id          0
visit_id            0
process_step        0
date_time           0
dtype: int64

In [190]:
#Eliminando nulos de las columnas
df_master = df_master.dropna(subset=["clnt_age","bal","logon_segment"])

In [191]:
#verificar que se modifica date_time
df_master["date_time"] = pd.to_datetime(df_master["date_time"])

In [192]:
df_master.duplicated().sum()

4072

In [193]:
df_master = df_master.drop_duplicates(
    subset=["visit_id","process_step","date_time"]
)

## OBJETIVO PRINCIPAL 

#La nueva interfaz hace que más clientes terminen el proceso online?
Eso se mide con:


tasa de finalización (conversion rate)\
= % de visitas que llegan a confirm.

# KPI´s Y MÉTRICAS

## Tasa de Finalización

In [194]:
#Ordernar por visit_id y date_time
df_master = df_master.sort_values(["visit_id","date_time"])

#Filtrar visitas válidas (empiezan en start)
first_step = (df_master.groupby("visit_id")["process_step"]
      .first().reset_index())

valid_visits = first_step[first_step["process_step"]=="start"]["visit_id"]

In [195]:
# Calcular conversión por visita 
completed = (
    df_master[df_master["visit_id"].isin(valid_visits)]
      .groupby(["visit_id","Variation"])["process_step"]
      .apply(lambda x: "confirm" in x.values)
      .reset_index(name="completed")
)

In [196]:
conversion_rate = completed.groupby("Variation")["completed"].mean()
conversion_rate

Variation
Control    0.481893
Test       0.540689
Name: completed, dtype: float64

In [197]:
#El nuevo diseño mejora significativamente la tasa de finalización.

## Tiempo dedicado a cada paso 

In [198]:
#ordenar df_master
df_master = df_master.sort_values(["visit_id","date_time"])

In [199]:
# creamos la columna next_time
df_master["next_time"] = (df_master.groupby("visit_id")["date_time"].shift(-1))

In [200]:
# crear time_spent (segundos)
df_master["time_spent"] = (df_master["next_time"] - df_master["date_time"]).dt.total_seconds()

In [201]:
df_time = df_master[df_master["time_spent"].notna()]

In [202]:
time_per_step = (df_time.groupby(["Variation","process_step"])["time_spent"].mean())

In [203]:
time_per_step

Variation  process_step
Control    confirm         180.183400
           start            66.817815
           step_1           50.546425
           step_2           92.052577
           step_3          137.206383
Test       confirm         250.566239
           start            61.471723
           step_1           60.756638
           step_2           88.877879
           step_3          129.620668
Name: time_spent, dtype: float64

## Tasas de Error 

In [204]:
# Mapear pasos
step_order = {
    "start": 0,
    "step_1": 1,
    "step_2": 2,
    "step_3": 3,
    "confirm": 4
}

df_master["step_num"] = df_master["process_step"].map(step_order)

# Ordenar
df_master = df_master.sort_values(["visit_id", "date_time"])

# Obtener paso anterior
df_master["prev_step"] = (df_master.groupby("visit_id")["step_num"].shift(1))

In [205]:
#Un error ocurre cuando el número de paso disminuye dentro de una misma visita.
df_master["error"] = df_master["step_num"] < df_master["prev_step"]

In [206]:
error_visit = (
    df_master.groupby(["visit_id","Variation"])["error"].any().reset_index())

In [207]:
#calculando tasa por grupo 
error_rate = (error_visit.groupby("Variation")["error"].mean())
error_rate

Variation
Control    0.205152
Test       0.270352
Name: error, dtype: float64

en el grupo test el 27.1% de las visitas tuvieron al menos un retroceso\
en control 20.6% tuvieron retroceso.

27.1% − 20.6% = +6.5 puntos porcentuales

### Resultado de rediseño

DIFERENCIA ABSOLUTA

Control = 0.206426\
Test = 0.271322

0.271322 - 0.206426 = 0.064896\
+6.46 puntos porcentuales

Mejora Relativa

0.064896 / 0.206426 ≈ 0.3144\
0.3144 × 100 ≈ 31.4%

Comparando Test vs Control:

Start
* Control: 62.9s
* Test: 60.6s\
Test ligeramente más rápido (~2s)

Step 1
* Control: 50.3s
* Test: 60.6s\
Test más lento (~10s)

Step 2
* Control: 91.6s
* Test: 88.6s\
Test ligeramente más rápido (~3s)

Step 3
* Control: 135.5s
* Test: 128.9s\
Test más rápido (~6s)

Confirm
* Control: 169.2s
* Test: 246.4s\
Test mucho más lento (~77s)

443.772 EVENTOS LIMPIOS, 20 COLUMNAS, SIN NULOS, SIN DUPLICADOS

# HIPÓTESIS 1

 TASA DE FINALIZACION 

Dentro de la definición de la hipótesis, estamos comparando dos proporciones: Control y Treatment 

Nuestra Hipotesis nula (Ho): No hay diferencia en la tasa de finalizacion entre control y test

H0​:p_test ​= p_control​

Hipótesis Alternativa (H1): El nuevo diseño tuvo mayor tasa

H1​:p_test​ > p_control​

In [129]:
#z-test
# Número de éxitos (confirmaciones)
success = (completed.groupby("Variation")["completed"].sum())

# Número total de visitas
n = (completed.groupby("Variation")["completed"].count())

success, n

(Variation
 Control    14864
 Test       17833
 Name: completed, dtype: int64,
 Variation
 Control    30845
 Test       32982
 Name: completed, dtype: int64)

In [130]:
#prueba estadistica central del A/B test
success_values = np.array([success["Test"], success["Control"]])

n_values = np.array([n["Test"], n["Control"]])

stat, p_value = proportions_ztest(success_values,n_values,alternative='larger')

print("Z-statistic:", stat)
print("p-value:", p_value)

Z-statistic: 14.850252581754853
p-value: 3.465351712919813e-50


Formalmente:
H0: p_test = p_control
H1: p_test > p_control

Dado que el p-value < 0.05, rechazamos la hipótesis nula y aceptamos la hipótesis alternativa.


In [131]:
#calculando intervalos
p_test = success_values[0] / n_values[0]
p_control = success_values[1] / n_values[1]

diff = (p_test - p_control).round(2)

print("Diferencia en tasa:", diff)
print("En puntos porcentuales:", diff * 100)

Diferencia en tasa: 0.06
En puntos porcentuales: 6.0


# HIPOTESIS 2 

 TAZA  DE FINALIZACION CON UN UMBRAL DE RENTABILIDAD 5%

Hipótesis Nula (H₀):\
El incremento en la tasa de finalización del grupo Test respecto al grupo Control es menor o igual al 5%.

H0: p_test - p_control =< 0.05

Hipótesis Alternativa (H₁):\
El incremento en la tasa de finalización del grupo Test respecto al grupo Control es mayor al 5%.

H1: p_test - p_control > 0.05

In [132]:
# Filtrar solo grupos del experimento
completed_group = completed[completed["Variation"].isin(["Control", "Test"])]

In [133]:
#Z-TEST (unilateral)
stat, p_value = proportions_ztest(success_values, n_values, alternative='larger')

print("Z-statistic:", stat)
print("p-value:", p_value)

Z-statistic: 14.850252581754853
p-value: 3.465351712919813e-50


In [134]:
#TASA Y LIFT
p_test = success_values[0] / n_values[0]
p_control = success_values[1] / n_values[1]

lift = (p_test - p_control).round(2)

print("\nTasa Control:", p_control)
print("Tasa Test:", p_test)
print("Lift absoluto:", lift)
print("Lift en puntos porcentuales:", lift * 100)


Tasa Control: 0.48189333765602205
Tasa Test: 0.5406888605906252
Lift absoluto: 0.06
Lift en puntos porcentuales: 6.0


In [135]:
# INTERVALO DE CONFIANZA
ci_low, ci_high = confint_proportions_2indep(
    success_values[0], n_values[0],   # Test
    success_values[1], n_values[1],   # Control
    method='wald')

print("\nIC 95% diferencia (Test - Control):", ci_low, ci_high)


IC 95% diferencia (Test - Control): 0.051048306217955246 0.06654273965125096


In [136]:
# DECISIÓN ESTADÍSTICA
alpha = 0.05

if p_value < alpha:
    print("\nResultado estadístico: Rechazamos H0. El nuevo diseño mejora la conversión.")
else:
    print("\nResultado estadístico: No hay evidencia suficiente para afirmar mejora.")

# DECISIÓN ECONÓMICA (UMBRAL 5%)

if lift >= 0.05:
    print("Cumple el umbral económico del 5% (estimación puntual).")
else:
    print("NO cumple el umbral económico del 5%.")



Resultado estadístico: Rechazamos H0. El nuevo diseño mejora la conversión.
Cumple el umbral económico del 5% (estimación puntual).


Se comparó la tasa de finalización entre el grupo control y el grupo test, el análisis A/B mostró un incremento del 2.5% en la tasa de finalización para el grupo test, pero este incremento no alcanzó el umbral de rentabilidad del 5%, por lo que no se recomienda implementar el nuevo diseño.
Sin embargo, la decisión de implementar el nuevo diseño no debe basarse unicamente en la estadística, si no también en si el incremento observado en la tasa de finalización alcanza o supera el umbral mínimo del 5% establecido por Vanguard.
El rediseño sólo se considera existoso desde una perspectiva de negocio si cumple este ceiterio de rentabilidad.

# HIPOTESIS 3

### TAZA DE ABANDONO 

H0: p_aband_test = p_aband_control\
H1: p_aband_test < p_aband_control

In [137]:
# Calcular abandono
completed_ab["abandoned"] = ~completed_ab["completed"]

# Tasa descriptiva
abandonment_rate = (completed_ab.groupby("Variation")["abandoned"].mean())

print("Tasa de abandono:")
print(abandonment_rate)

Tasa de abandono:
Variation
Control    0.518107
Test       0.459311
Name: abandoned, dtype: float64


In [138]:
# Éxitos = abandonos
ab_counts = completed_ab.groupby("Variation")["abandoned"].sum()
ab_total = completed_ab.groupby("Variation")["abandoned"].count()

counts = np.array([ab_counts["Test"], ab_counts["Control"]])
nobs = np.array([ab_total["Test"], ab_total["Control"]])

# Total visitas
nobs = np.array([
    completed_ab.groupby("Variation")["abandoned"].count()["Test"],
    completed_ab.groupby("Variation")["abandoned"].count()["Control"]
])

# Prueba unilateral: ¿Test < Control?
stat, p_value = proportions_ztest(counts, nobs, alternative='smaller')

print("\nZ-stat:", stat)
print("p-value:", p_value)


Z-stat: -14.850252581754868
p-value: 3.46535171291907e-50


El nuevo diseño reduce significativamente la tasa de abandono.\
Es coherente con la hipotesis 1, que la conversion aumenta +5.8 pp

(0.459311 - 0.518107) * 100 = y el abandono disminuye -5.88

La tasa de abandono es la proporcion de usuarios que inician el proceso pero no llegan al paso final de confirmación, es decir, los usuarios que tienen al menos un process_step pero no llegan al paso final.Esto se puede interpretar por fricción, confusion, fatiga en el proceso u otras variables .

# HIPOTESIS 4
EL EFECTO DEL REDISEÑO ES MAYOR EN CLIENTES JÓVENES

Ho: El efecto del rediseño es igual en todos los grupos de edad.
(No hay edad de interacción X Variation)
Binteraction = 0

H1:
El efecto del rediseño es mayor en clientes jòvenes
Binteration =/ 0

In [139]:
#Generando variable de finalización 
completed = (
    df_master
      .groupby(["visit_id","Variation","clnt_age"])["process_step"]
      .apply(lambda x: "confirm" in x.values)
      .reset_index(name="completed")
)


In [140]:
#Creando grupo de edad 
age_median = completed["clnt_age"].median()

completed["young"] = completed["clnt_age"] < age_median


young = True --> jóvenes
young = False --> mayores 

In [141]:
#Verificando tasas simples 
completed.groupby(["young","Variation"])["completed"].mean()


young  Variation
False  Control      0.457069
       Test         0.545975
True   Control      0.541551
       Test         0.625886
Name: completed, dtype: float64

In [142]:
#Convirtiendo completed a entero y young a numérica 
completed["completed"] = completed["completed"].astype(int)
completed["young"] = completed["young"].astype(int)


In [143]:
completed["Variation"] = completed["Variation"].astype("category")


In [144]:
#TEST A/B
#Test con regresión logística con interacción

model = smf.logit(
    "completed ~ Variation + young + Variation:young",
    data=completed
).fit()

print(model.summary())

Optimization terminated successfully.
         Current function value: 0.681813
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:              completed   No. Observations:                69409
Model:                          Logit   Df Residuals:                    69405
Method:                           MLE   Df Model:                            3
Date:                Tue, 17 Feb 2026   Pseudo R-squ.:                 0.01047
Time:                        15:12:08   Log-Likelihood:                -47324.
converged:                       True   LL-Null:                       -47825.
Covariance Type:            nonrobust   LLR p-value:                7.657e-217
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                  -0.1721      0.016    -10.954      0.000      -0.203   

In [145]:
df_master.to_csv("df_master.csv", index=False)

## Evaluación del experimento 

### Eficacia del diseño

In [146]:
# a) Hubo asignación equilibrada?

df_exp_clients["Variation"].value_counts(normalize=True)

Variation
Test       0.53402
Control    0.46598
Name: proportion, dtype: float64

### La asignación esta equilibrada

In [147]:
# b) Se dividieron aleatoriamente y en partes iguales?
df_static.groupby("Variation")[["clnt_age","bal","clnt_tenure_yr"]].mean()

,clnt_age,bal,clnt_tenure_yr
Variation,,,
Control,47.498427,150151.900186,12.087860
Test,47.163922,148962.605032,11.982901


In [148]:
stats.ttest_ind(
    df_static[df_static["Variation"]=="Test"]["clnt_age"],
    df_static[df_static["Variation"]=="Control"]["clnt_age"],
    equal_var=False)

TtestResult(statistic=-2.416068061200627, pvalue=0.015692719461388685, df=49556.71518942814)

p-value < 0.05 -> posible sesgo de selección

#### se dividieron aleatoriamente 

In [149]:
df_static.groupby("Variation")[["clnt_age","bal","clnt_tenure_yr"]].mean()

stats.ttest_ind(
    df_static[df_static["Variation"]=="Test"]["clnt_age"],
    df_static[df_static["Variation"]=="Control"]["clnt_age"],
    equal_var=False
)

TtestResult(statistic=-2.416068061200627, pvalue=0.015692719461388685, df=49556.71518942814)

El hecho de que p < 0.05 no significa que la asignación no fue aleatoria.

Significa que:
* Hay una diferencia detectable.
* Pero probablemente es pequeña.
* No necesariamente implica sesgo sistemático.

### ¿Hubo algún sesgo?

In [150]:
df_master.groupby("Variation")["visit_id"].nunique()

Variation
Control    32181
Test       37122
Name: visit_id, dtype: int64

In [151]:
df_static["Variation"].value_counts(normalize=True)
df_master["Variation"].value_counts(normalize=True)

Variation
Test       0.557011
Control    0.442989
Name: proportion, dtype: float64

Si cambian mucho las proporciones → posible sesgo por limpieza.

#### Sesgo Por duración

In [152]:
print(df_master.groupby("Variation")["date_time"].min())
df_master.groupby("Variation")["date_time"].max()

Variation
Control   2017-03-15 00:19:28
Test      2017-03-15 00:43:23
Name: date_time, dtype: datetime64[ns]


Variation
Control   2017-06-20 23:57:06
Test      2017-06-20 23:21:23
Name: date_time, dtype: datetime64[ns]

No existe sesgo temporal observable.

Duración total ≈ 3 meses (≈ 97 días).

Eso es suficiente para:
* Capturar múltiples ciclos semanales.
* Evitar sesgo por comportamiento puntual.
* Obtener tamaño muestral robusto (como confirma tu p-value extremadamente pequeño).

La duración fue adecuada para obtener resultados estadísticamente sólidos.

#### ¿Qué otros datos, si estuvieran disponibles, podrían mejorar el análisis?

* Ingresos reales por conversión, para medir impacto económico y no solo conversión.
* Datos de retención o comportamiento posterior, para evaluar la calidad de la conversión.
* Información sobre dispositivo (móvil vs desktop), ya que el efecto del rediseño puede variar según el canal.
* Confirmación de exposición real al tratamiento, para evitar sesgos de implementación.

#### Bono: 

#### Análisis del comportamiento del cliente:

Siguen los mismos pasos o cambian el recorrido?

H0: número de pasos es igual\
H1: número de pasos es diferente

In [153]:
steps_per_visit = (
    df_master
    .groupby(["visit_id","Variation"])["process_step"]
    .count()
    .reset_index(name="n_steps")
)

steps_per_visit.groupby("Variation")["n_steps"].mean()

Variation
Control    4.365371
Test       4.758391
Name: n_steps, dtype: float64

In [154]:
control_steps = steps_per_visit[steps_per_visit["Variation"]=="Control"]["n_steps"]
test_steps = steps_per_visit[steps_per_visit["Variation"]=="Test"]["n_steps"]

stats.ttest_ind(test_steps, control_steps, equal_var=False)

TtestResult(statistic=18.438860235267885, pvalue=9.751614090830638e-76, df=68997.0913919645)

rechazamos la hipotesis nula, aceptamos la alternativa\
que es que hay diferencia significativa en número de pasos.

#### Eficiencia del proceso (tiempo total por visita)

¿El nuevo diseño reduce el tiempo total necesario para completar el proceso?

H0: el tiempo promedio total por visita es igual en ambos grupos\
H0: media_test = media_control

H1: El tiempo promedio total por visita es menor en el grupo test\ 
H1: media_test < media_control

In [155]:
# Tiempo total por visita
total_time_visit = (
    df_master
      .groupby(["visit_id","Variation"])["time_spent"]
      .sum()
      .reset_index(name="total_time_sec")
)

total_time_visit.head()

,visit_id,Variation,total_time_sec
0,100012776_37918976071_457913,Test,52.0
1,100019538_17884295066_43909,Test,242.0
2,100022086_87870757897_149620,Test,180.0
3,100030127_47967100085_936361,Control,0.0
4,100037962_47432393712_705583,Control,132.0


In [156]:
total_time_visit.groupby("Variation")["total_time_sec"].agg(["mean","median","std"])

,mean,median,std
Variation,,,
Control,282.304217,160.0,532.058866
Test,317.764533,168.0,594.699970


In [157]:
control_time = total_time_visit[
    total_time_visit["Variation"]=="Control"
]["total_time_sec"]

test_time = total_time_visit[
    total_time_visit["Variation"]=="Test"
]["total_time_sec"]

t_stat, p_value = stats.ttest_ind(test_time, control_time, equal_var=False)

print("t-stat:", t_stat)
print("p-value:", p_value)

t-stat: 8.283880955696898
p-value: 1.2132782955586003e-16


p < 0.05 y media_test < media_control\
→ El nuevo diseño es significativamente más eficiente.

#### Tamaño del efecto (Effect Size)

In [158]:
p1 = p_test
p2 = p_control

h = 2 * (np.arcsin(np.sqrt(p1)) - np.arcsin(np.sqrt(p2)))
print("Cohen's h:", h)

Cohen's h: 0.11768905272372487


In [159]:
# tamaño del efecto real del rediseño

#### Potencia post-hc (Post-hoc Power)

In [160]:
analysis = NormalIndPower()

power = analysis.power(
    effect_size=h,
    nobs1=n_values[0],
    alpha=0.05,
    ratio=n_values[1]/n_values[0]
)

print("Power:", power)

Power: 1.0


In [161]:
# el experimento estuvo sobredimensionado

#### Análisis de potencia (fase de diseño)

In [162]:
analysis = NormalIndPower()

required_n = analysis.solve_power(
    effect_size=0.2,  # mínimo efecto detectable
    alpha=0.05,
    power=0.8,
    ratio=1
)

print("Muestra requerida por grupo:", required_n)

Muestra requerida por grupo: 392.4430232577885


In [163]:
effect_size = 2 * (np.arcsin(np.sqrt(0.53)) - np.arcsin(np.sqrt(0.48)))

required_n = analysis.solve_power(
    effect_size=effect_size,
    alpha=0.05,
    power=0.80,
    ratio=1
)

print("Tamaño mínimo por grupo:", np.ceil(required_n))

Tamaño mínimo por grupo: 1569.0


El tamaño mínimo por grupo necesario, es de 1569.